In [30]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import json


from torch.utils.data import ConcatDataset, DataLoader, Subset
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from Helpers.data import SDDataset, PointCloudDataset

if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

Using: cuda


In [26]:
torch.save(trainset, '../Data/sd_dataset_3072_train_easier')
torch.save(testset, '../Data/sd_dataset_3072_test_easier')

In [28]:
trainset = torch.load('../Data/sd_dataset_3072_train_easier.pt', weights_only= False)
testset = torch.load('../Data/sd_dataset_3072_test_easier.pt', weights_only= False)
# trainset = SDDataset('../Data/ModelNet40', '../Data/sampled_points_easier', device, 3072, 'train', object_classes= None)
# testset = SDDataset('../Data/ModelNet40', '../Data/sampled_points_easier', device, 3072, 'test', object_classes= None)

batch_size = 32

train_loader = DataLoader(trainset, batch_size= batch_size, shuffle= True)
test_loader = DataLoader(testset, batch_size= batch_size, shuffle= True)


In [23]:
class SD_Model_Simple(nn.Module):

    def __init__(self, latent_rep_size):
        
        super().__init__()

        input_size = 3 + latent_rep_size
        self.l1 = nn.Linear(input_size, 1024)
        self.l2 = nn.Linear(1024, 2048)
        self.l3 = nn.Linear(2048, 1024)
        self.l4 = nn.Linear(1024, 128)
        self.l5 = nn.Linear(128, 1)

    def forward(self, latent_rep, xyz):
        x = torch.concat((latent_rep, xyz), dim = 1)
        x = F.gelu(self.l1(x))
        x = F.gelu(self.l2(x))
        x = F.gelu(self.l3(x))
        x = F.gelu(self.l4(x))
        x = self.l5(x)

        return x
    
class SD_Model_Complex(nn.Module):

    def __init__(self, latent_rep_size):
        
        super().__init__()

        self.up1 = nn.Linear(3, 64)
        self.up2 = nn.Linear(64, 128)
        self.up3 = nn.Linear(128, latent_rep_size)

        self.l1 = nn.Linear(latent_rep_size * 2, 4096)
        self.l2 = nn.Linear(4096, 2048)
        self.l3 = nn.Linear(2048, 1024)
        self.l4 = nn.Linear(1024, 512)
        self.l5 = nn.Linear(512, 64)
        self.l6 = nn.Linear(64, 1)

        self.up_bn1 = nn.BatchNorm1d(64)
        self.up_bn2 = nn.BatchNorm1d(128)
        self.up_bn3 = nn.BatchNorm1d(latent_rep_size)
        
        self.l_bn1 = nn.BatchNorm1d(4096)
        self.l_bn2 = nn.BatchNorm1d(2048)
        self.l_bn3 = nn.BatchNorm1d(1024)
        self.l_bn4 = nn.BatchNorm1d(512)
        self.l_bn5 = nn.BatchNorm1d(64)


    def forward(self, latent_rep, xyz):
        xyz = F.gelu(self.up_bn1(self.up1(xyz)))
        xyz = F.gelu(self.up_bn2(self.up2(xyz)))
        xyz = self.up3(xyz)

        x = torch.concat((latent_rep, xyz), dim = 1)

        x = F.gelu(self.l_bn1(self.l1(x)))
        x = F.gelu(self.l_bn2(self.l2(x)))
        x = F.gelu(self.l_bn3(self.l3(x)))
        x = F.gelu(self.l_bn4(self.l4(x)))
        x = F.gelu(self.l_bn5(self.l5(x)))
        x = self.l6(x)

        return x
    

class SD_Model_Small(nn.Module):

    def __init__(self, latent_rep_size):
        
        super().__init__()

        self.up1 = nn.Linear(3, 64)
        self.up2 = nn.Linear(64, latent_rep_size)

        self.l1 = nn.Linear(latent_rep_size * 2, 2048)
        self.l2 = nn.Linear(2048, 1024)
        self.l3 = nn.Linear(1024, 512)
        self.l4 = nn.Linear(512, 256)
        self.l5 = nn.Linear(256, 1)

        self.up_bn1 = nn.BatchNorm1d(64)
        
        self.l_bn1 = nn.BatchNorm1d(2048)
        self.l_bn2 = nn.BatchNorm1d(1024)
        self.l_bn3 = nn.BatchNorm1d(512)
        self.l_bn4 = nn.BatchNorm1d(256)


    def forward(self, latent_rep, xyz):
        xyz = F.gelu(self.up_bn1(self.up1(xyz)))
        xyz = self.up2(xyz)

        x = torch.concat((latent_rep, xyz), dim = 1)

        x = F.gelu(self.l_bn1(self.l1(x)))
        x = F.gelu(self.l_bn2(self.l2(x)))
        x = F.gelu(self.l_bn3(self.l3(x)))
        x = F.gelu(self.l_bn4(self.l4(x)))
        x = self.l5(x)

        return x

In [32]:
def train_model(key, model, num_epochs, train_loader, val_loader):

    optimizer = optim.AdamW(model.parameters(), lr=0.0001)

    min_val_loss = np.inf

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):

        # Train one epoch
        train_loss = 0 
        
        count = 0
        for data in tqdm(train_loader):
            
            latent_rep = data['latent_rep'].to(device)
            xyz = data['xyz'].to(device)
            sd = (data['sd'] > 0).float().to(device)

            pred = model(latent_rep, xyz)
            pred = pred.squeeze()

            optimizer.zero_grad()
            
            loss = F.binary_cross_entropy_with_logits(pred, sd)

            loss.backward()

            optimizer.step()

            train_loss += loss.item()

            if count % 50000 == 0:
                print(f'{count}: {loss.item()}')

            count +=1

        train_loss /= len(train_loader)

        # Calculate validation loss

        val_loss = 0 

        for data in val_loader:

            latent_rep = data['latent_rep'].to(device)
            xyz = data['xyz'].to(device)
            # sd = data['sd'].to(device)
            sd = (data['sd'] >  0).float().to(device)

            with torch.no_grad():
                pred = model(latent_rep, xyz)
                pred = pred.squeeze()

                loss = F.binary_cross_entropy_with_logits(pred, sd)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f'\nEpoch {epoch+1} \t Train Loss: {train_loss:.5f} \t Val Loss: {val_loss:.5f}')

        # Save best model
        if val_loss < min_val_loss:
            print(f'Val Loss Decreased({min_val_loss:.6f} ---> {val_loss:.6f}) \t Saving The Model')
            min_val_loss = val_loss

            torch.save(model.state_dict(), f'./trained_sdf_models/{key}')

        train_losses.append(train_loss)
        val_losses.append(val_loss)

    return train_losses, val_losses

model = SD_Model_Complex(512)
model = model.to(device)

num_epochs = 3
key = "mlp_512_complex"
train_losses, val_losses = train_model(key, model, num_epochs , train_loader, test_loader)

results = {
    'train_losses': train_losses,
    'val_losses' : val_losses
           }

with open(f'./{key}_results.json', 'w') as f: 
    json.dumps(f, results)


  0%|          | 1/3072813 [00:03<3049:38:22,  3.57s/it]

0: 0.7931326031684875


  0%|          | 5037/3072813 [00:23<3:09:05, 270.39it/s]

5000: 0.6775326728820801


  0%|          | 10034/3072813 [00:42<3:11:57, 265.92it/s]

...

55000: 0.4434249699115753


  2%|▏         | 60027/3072813 [03:52<3:32:09, 236.68it/s]

60000: 0.505903959274292


  2%|▏         | 64689/3072813 [04:12<3:15:25, 256.55it/s]


KeyboardInterrupt: 

In [ ]:
model = SD_Model_Simple(512)
model = model.to(device)

num_epochs = 3
key = "mlp_512_simple"
train_losses, val_losses = train_model(key, model, num_epochs , train_loader, test_loader)

results = {
    'train_losses': train_losses,
    'val_losses' : val_losses
           }

with open(f'./{key}_results.json', 'w') as f: 
    json.dumps(f, results)


In [ ]:
model = SD_Model_Small(512)
model = model.to(device)

num_epochs = 3
key = "mlp_512_small"
train_losses, val_losses = train_model(key, model, num_epochs , train_loader, test_loader)

results = {
    'train_losses': train_losses,
    'val_losses' : val_losses
           }

with open(f'./{key}_results.json', 'w') as f: 
    json.dumps(f, results)